# Astigmatism Test

Author: A. W. Jones

In this test we will set up a astigmatic beam into a fabry perot cavity and check that the response matches the response in [1]. In order to check that we will first check:
  - The g factor is computed correctly
  - Inputing an mode mismatched TEM00 beam results in an excitation of a HG20 cavity mode
  - Higher order modes resonate at the expected position

Finnaly, we will check that the thoughput of an astigmaic input beam, through a stigmatic cavity aginst the results in [1]. This checks that astigmatism is being properly moddelled for several higher order modes.

The tolerance is pretty high on the final check, this is because the analytics are only valid for a small mismatch and 

[1] A. W. Jones and A. Freise, "Increased sensitivity of higher-order laser beams to mode mismatches," Opt. Lett. 45, 5876-5878 (2020)

## Setup

In [ ]:
# std lib
from copy import deepcopy
import sys; import os

# scipy
import numpy as np
import matplotlib.pyplot as plt

# third party
import finesse
from finesse import BeamParam

# Hacks to plots to BEST
if 'BESTDATASERVER' in os.environ:
    from bestutils import do_upload
else:
    def do_upload(*args,**kwargs): pass

Diagram:
```
Laser <-- 2m --> m1 <-- 1.7m --> m2 <-- 1m --> pdtrans
```

In [ ]:
L = 1.7
Rc1 = Rc2 = 1
g1 = 1-L/Rc1
g2 = 1-L/Rc2
g = g1*g2
print(f'g-factor {g}')

IFO = finesse.Model()
IFO.parse(f"""

l LaserIn P=1

s s0 LaserIn.p1 ITM.p1 L=1

m ITM R=0.99 T=0.01 Rc={-Rc1}
s sCAV ITM.p2 ETM.p1 L={L}
m ETM R=0.99 T=0.01 Rc={Rc2}

cavity fab1 ITM.p2 via=ETM.p1.i
power_detector_dc Transmitted_Power ETM.p2.o
power_detector_dc Circulating_Power ETM.p1.o
""")

IFO.select_modes(maxtem=4)

try:
    assert np.isclose(g,IFO.fab1.gx,atol=1e-10,rtol=0)
    assert np.isclose(g,IFO.fab1.gy,atol=1e-10,rtol=0)
except AssertionError:
    print(f'Analytical g-factor: [{g}, {g}] does not match Finesse computation {IFO.fab1.g}')
    sys.exit(1)
    
trace = IFO.beam_trace()
bpx_Laser, bpy_Laser = trace[IFO.LaserIn.p1.o]
print(f'Beam parameter at laser: waist = {1e6*bpx_Laser.w0:.1f}'
    +f'um, z = {bpx_Laser.z:.3f}')
try: 
    assert np.isclose(bpx_Laser.w0,bpy_Laser.w0,atol=1e-10,rtol=0)
    assert np.isclose(bpx_Laser.z,bpy_Laser.z,atol=1e-10,rtol=0)
except AssertionError:
    print(f'Beam is astigmatic in cylindracally symetic cavity')
    print(f'bpx w0: {bpx_Laser.w0}, bpy w0: {bpy_Laser.w0}')
    print(f'bpx z: {bpx_Laser.z}, bpy z: {bpy_Laser.z}')
    sys.exit(1)

## Test that an astigmatic beam results in HG20 being excited

In [ ]:
_test1 = deepcopy(IFO)
_test1.parse('xaxis ETM.phi lin -1 181 10000')
_test1.parse(f'gauss gLaser LaserIn.p1.o w0={0.95*bpx_Laser.w0} z={1*bpx_Laser.z}')
_test1.parse(f'amplitude_detector circ20 ETM.p1.i 0 n=2 m=0')
out = _test1.run()
out.plot(logy=True)

fname = 'test_astigmatic_beam_exicites_HG20.png'
plt.savefig(fname,dpi=300,bbox_inches='tight')
do_upload(fname)

# near the resonance 
try: 
    assert np.max(np.abs(out['circ20'])) > 0.1 
except AssertionError:
    print(f'HG20 mode expected at the 10% level when beam is mismatched to cavity')
    print(f'Maximum amplitude during cav scan: {np.max(np.abs(out["circ20"]))}')
    sys.exit(1)


## Test power can be moved into a higher order mode

In [ ]:
def res_position(n,m):
    """ From Eq 9.36 in _Interferometer Techniques 
    for Gravitational-Wave Detection_ by Bond et. al."""
    return 180 - ((n*IFO.fab1.gouy_x + m*IFO.fab1.gouy_y)/2 % 180)

_test2 = deepcopy(IFO)
_test2.parse('xaxis ETM.phi lin -1 181 10000')

# Set laser powers
# If these add up to one
# then this is the power 
# each mode. Otherwise
# Finesse does some rescaling
# such that the total power
# is conserved
set_powers = [0.4,0.35,0.25]
assert np.sum(set_powers) == 1

_test2.LaserIn.tem(0,0,set_powers[0])
_test2.LaserIn.tem(0,1,set_powers[1])
_test2.LaserIn.tem(2,0,set_powers[2])

out = _test2.run()
out.plot(logy=True)

fname = 'test_power_moved_into_hom.png'
plt.savefig(fname,dpi=300,bbox_inches='tight')
do_upload(fname)

x, = out.x

idx = np.argmin(np.abs(x))
pout = []
pout.append(out['Transmitted_Power'][idx])

idx = np.argmin(np.abs(x - res_position(0,1)))
pout.append(out['Transmitted_Power'][idx])

idx = np.argmin(np.abs(x - res_position(2,0)))
pout.append(out['Transmitted_Power'][idx])

try: 
    assert np.all(np.isclose(set_powers,pout,rtol=0,atol=1e-4))
except AssertionError:
    print(f'Failed check that power is correctly moved into'
          ' higher order modes. This could either be the '
          ' mode resonating in the `wrong` place, or the tem'
          ' command not working.')
    print(f'Input: {set_powers}, output: {pout}')
    sys.exit(1)


## Test Throughput of Astigmatic Higher Order Modes

Test against Equation 9 in [1].

In [ ]:
def W(w):
    return 0.25*((w-1)**2 - (w-1)**3)

def Cn(n):
    if n==0: return 1
    elif n==1: return 3
    elif n==2: return 7
    elif n==3: return 13
    elif n==4: return 21
    else:
        raise NotImplementedError

def power_coupling(wx,wy,n,m): return 1 - Cn(n)*W(wx) - Cn(m)*W(wy)

# This needs to be one so that
# we dont need to rescale the output
# to throughput.
# If this fails you need to 
# (IFO.LaserIn.P - ptrans) / IFO.LaserIn.P
assert IFO.LaserIn.P.value == 1 

modes = [(0,0), (1,1), (2,1)]
win = np.linspace(0.95,1.05,num=10)
ptrans = {}
analytic = {}
wy = 0.97
for n,m in modes:
    
    # find resonance position
    _res = res_position(n,m)
    ptrans[n,m] = []
    analytic[n,m] = []
    for wx in win:
    
        _ifo = deepcopy(IFO)

        #scan a little either side of the resonance to find the peak
        _ifo.parse(f'xaxis ETM.phi lin {_res-1} {_res+1} 1000')

        # Set power in the modes
        _ifo.LaserIn.tem(0,0,0)
        _ifo.LaserIn.tem(n,m,1)
        _ifo.parse(f'gauss gLaser LaserIn.p1.o '
                   f'w0x={wx*bpx_Laser.w0} zx={bpx_Laser.z} '
                   f'w0y={wy*bpx_Laser.w0} zy={bpx_Laser.z} ')
        
        out = _ifo.run()
        ptrans[n,m].append(np.max(out['Transmitted_Power']))
        analytic[n,m].append(power_coupling(wx,wy,n,m))

    ptrans[n,m] = np.array(ptrans[n,m])
    analytic[n,m] = np.array(analytic[n,m])

In [ ]:
fig, ax = plt.subplots(nrows=2,sharex=True,
                      figsize=(7,6))

for n,m in modes:
    loss = 1 - ptrans[n,m]
    error = ptrans[n,m] - analytic[n,m]
    
    ax[0].plot(win,1e6*loss,label=f'n={n}, m={m}')
    ax[1].plot(win,1e6*error)

for _ax in ax:
    _ax.grid(True)
ax[0].legend(title='Input Mode')
ax[0].set_title('Mode mismatching for several modes')
ax[0].set_ylabel('Mode Matching Loss\n[ppm]')
ax[1].set_ylabel('Error (Finesse - Analytic)\n[ppm]')

#fig.tight_layout()
file = 'astigmatism_and_homs_test.png'
fig.savefig(file,bbox_inches='tight',dpi=300)

do_upload(file)

In [ ]:
for n,m in modes:
    try: 
        assert np.all(np.isclose(ptrans[n,m],analytic[n,m],rtol=1e-2,atol=1e-4))
    except AssertionError:
        print(f'Thoughput of higher order astigamtic beam did not match  arXiv:2007.12564')
        print(f'See plot for more details')
        sys.exit(10)